In [2]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client

c = Client()
c

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:41288 Dashboard: http://127.0.0.1:35408/status,Cluster Workers: 8 Cores: 48 Memory: 134.91 GB


In [3]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
import datetime
import pandas as pd
import sys

today=datetime.date.today()

import sys
sys.path.insert(0,'/home/albert7a/git/xscale')
import xscale

In [4]:
data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02-S/'
gridfile='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/mesh_mask_eNATL60_3.6_lev1.nc4'
dsgrid=xr.open_dataset(gridfile,chunks={'x':1000,'y':1000})


In [5]:
def read_csv(box):
    boxes=pd.read_csv('/home/albert7a/git/formation_ANNA/make_boxes/boxes_'+str(box)+'_1x1_eNATL60.csv',sep = '\t',index_col=0)
    imin=boxes['imin']
    imax=boxes['imax']
    jmin=boxes['jmin']
    jmax=boxes['jmax']
    box_name=boxes.index
    return imin,imax,jmin,jmax,box_name

def compute_buoy(t,s):
    rau0  = 1000
    grav  = 9.81
    buoy= -1*(grav/rau0)*sigma0(t,s)
    return buoy

def sigma0(t,s):
    zrau0=1000
    zsr=np.sqrt(np.abs(s))
    zs=s
    zt=t
    zr1 = ( ( ( ( 6.536332e-9*zt-1.120083e-6 )*zt+1.001685e-4)*zt - 9.095290e-3 )*zt+6.793952e-2 )*zt+999.842594
    zr2= ( ( ( 5.3875e-9*zt-8.2467e-7 )*zt+7.6438e-5 ) *zt - 4.0899e-3 ) *zt+0.824493
    zr3= ( -1.6546e-6*zt+1.0227e-4 ) *zt-5.72466e-3
    zr4= 4.8314e-4
    sigma0=( zr4*zs + zr3*zsr + zr2 ) *zs + zr1 - zrau0
    return sigma0

def filt(w):
    win_box2D = w.window
    win_box2D.set(window='hanning', cutoff=20, dim=['x', 'y'], n=[30, 30])
    bw = win_box2D.boundary_weights(drop_dims=[])
    w_LS = win_box2D.convolve(weights=bw)
    w_SS=w-w_LS
    return w_SS


In [6]:
def compute_wprime_bprime(date):
    print('read the data')
    tfilename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridT_'+date+'-'+date+'.nc'))
    tfile=tfilename[0]
    dst=xr.open_dataset(tfile,chunks={'x':1000,'y':1000,'time_counter':1,'deptht':1})
    tdata=dst['votemper']
    sfilename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridS_'+date+'-'+date+'.nc'))
    sfile=sfilename[0]
    dss=xr.open_dataset(sfile,chunks={'x':1000,'y':1000,'time_counter':1,'deptht':1})
    sdata=dss['vosaline']
    wfilename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridW_'+date+'-'+date+'.nc'))
    wfile=wfilename[0]
    dsw=xr.open_dataset(wfile,chunks={'x':1000,'y':1000,'time_counter':1,'depthw':1})
    wdata=dsw['vovecrtz']
    wdata_t=wdata.rename({'depthw':'deptht'})
    print('compute buoyancy')
    buoy=compute_buoy(tdata,sdata)
    print('filter w')
    wprime=filt(wdata_t)
    print('filter buoyancy')
    bprime=filt(buoy)
    wprimebprime=wprime*bprime
    return wprimebprime

In [7]:
date='20090701'
print('read the data')
tfilename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridT_'+date+'-'+date+'.nc'))
tfile=tfilename[0]
dst=xr.open_dataset(tfile,chunks={'x':1000,'y':1000,'time_counter':1,'deptht':1})
tdata=dst['votemper']
sfilename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridS_'+date+'-'+date+'.nc'))
sfile=sfilename[0]
dss=xr.open_dataset(sfile,chunks={'x':1000,'y':1000,'time_counter':1,'deptht':1})
sdata=dss['vosaline']
wfilename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridW_'+date+'-'+date+'.nc'))
wfile=wfilename[0]
dsw=xr.open_dataset(wfile,chunks={'x':1000,'y':1000,'time_counter':1,'depthw':1})
wdata=dsw['vovecrtz']
wdata_t=wdata.rename({'depthw':'deptht'})
print('compute buoyancy')
buoy=compute_buoy(tdata,sdata)
print('filter w')
wprime=filt(wdata_t)
print('filter buoyancy')
bprime=filt(buoy)


read the data
compute buoyancy
filter w


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:34254 remote=tcp://127.0.0.1:41288>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:34474 remote=tcp://127.0.0.1:41288>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:34584 remote=tcp://127.0.0.1:41288>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:34588 remote=tcp://127.0.0.1:41288>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:34594 remote=tcp://127.0.0.1:41288>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:34600 remote=tcp://127.0.0.1:41288>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:34610 remote=tcp://127.0.0.1:41288>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:34612 remote=tcp://127.0.0.1:41288>
distribu

filter buoyancy


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:41242 remote=tcp://127.0.0.1:41288>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:41314 remote=tcp://127.0.0.1:41288>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:41322 remote=tcp://127.0.0.1:41288>
distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)


In [14]:
wdata

<xarray.DataArray 'vovecrtz' (time_counter: 24, depthw: 300, y: 4729, x: 8354)>
dask.array<shape=(24, 300, 4729, 8354), dtype=float32, chunksize=(1, 1, 1000, 1000)>
Coordinates:
    nav_lat        (y, x) float32 dask.array<shape=(4729, 8354), chunksize=(1000, 1000)>
    nav_lon        (y, x) float32 dask.array<shape=(4729, 8354), chunksize=(1000, 1000)>
  * depthw         (depthw) float32 0.0 1.0000261 2.156828 ... 5949.999 6000.0
    time_centered  (time_counter) datetime64[ns] dask.array<shape=(24,), chunksize=(1,)>
  * time_counter   (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2009-07-01T23:30:00
Dimensions without coordinates: y, x
Attributes:
    standard_name:       upward_sea_water_velocity
    long_name:           ocean vertical velocity
    units:               m/s
    online_operation:    average
    interval_operation:  40 s
    interval_write:      1 h
    cell_methods:        time: mean (interval: 40 s)

In [25]:
deptht=dst.deptht
wdata_t=wdata.reindex({'depthw':deptht},method='nearest')
wdata_t=wdata_t.rename({'depthw':'deptht'})

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/core/dataarray.py:979: FutureWarning: Indexer has dimensions ('deptht',) that are different from that to be indexed along depthw. This will behave differently in the future.
  indexers=indexers, method=method, tolerance=tolerance, copy=copy)


In [26]:
wprime=filt(wdata_t)


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:40702 remote=tcp://127.0.0.1:41288>
distributed.utils_perf - WARNING - full garbage collections took 36% CPU time recently (threshold: 10%)


In [27]:
wprime

<xarray.DataArray 'vovecrtz' (time_counter: 24, deptht: 300, y: 4729, x: 8354)>
dask.array<shape=(24, 300, 4729, 8354), dtype=float64, chunksize=(1, 1, 1000, 1000)>
Coordinates:
  * deptht         (deptht) float32 0.48045477 1.5587916 ... 5974.8853 6025.346
    nav_lat        (y, x) float32 dask.array<shape=(4729, 8354), chunksize=(1000, 1000)>
    nav_lon        (y, x) float32 dask.array<shape=(4729, 8354), chunksize=(1000, 1000)>
    time_centered  (time_counter) datetime64[ns] dask.array<shape=(24,), chunksize=(1,)>
  * time_counter   (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2009-07-01T23:30:00
Dimensions without coordinates: y, x

In [28]:
wprimebprime=wprime*bprime

In [29]:
wprimebprime

<xarray.DataArray (time_counter: 24, deptht: 300, y: 4729, x: 8354)>
dask.array<shape=(24, 300, 4729, 8354), dtype=float64, chunksize=(1, 1, 1000, 1000)>
Coordinates:
  * deptht         (deptht) float32 0.48045477 1.5587916 ... 5974.8853 6025.346
    nav_lat        (y, x) float32 dask.array<shape=(4729, 8354), chunksize=(1000, 1000)>
    nav_lon        (y, x) float32 dask.array<shape=(4729, 8354), chunksize=(1000, 1000)>
    time_centered  (time_counter) datetime64[ns] dask.array<shape=(24,), chunksize=(1,)>
  * time_counter   (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2009-07-01T23:30:00
Dimensions without coordinates: y, x

In [ ]:
dataset=wprimebprime.to_dataset(name='wprimebprime')
attrs=tdata.attrs
dataset['wprimebprime'].attrs=attrs
dataset['wprimebprime'].attrs['standard_name']='product of filtered vertical velocity and buoyancy'
dataset['wprimebprime'].attrs['long_name']='wprimebprime'
dataset['wprimebprime'].attrs['units']=''        
dataset.attrs['global_attribute']= 'product of filtered vertical velocity and buoyancy'
dataset.to_netcdf(path='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/ANNA/eNATL60-BLBT02_y'+date[0:4]+'m'+date[4:6]+'d'+date[6:9]+'_wprimebprime.nc',mode='w')
